# © Artur Czarnecki. All rights reserved.
# Integrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.


In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../..")))

import sys
from pathlib import Path

from intergrax.supervisor.supervisor import IntergraxSupervisor, SupervisorConfig, SupervisorPromptPack
from intergrax.llm_adapters import LLMAdapterRegistry
from intergrax.rag.embedding_manager import IntergraxEmbeddingManager
from langchain_ollama import ChatOllama
from components.compliance_checker import compliance_checker
from components.cost_estimator import cost_estimator
from components.final_summary import final_summary
from applications.ux_audit_agent.components.general_knowledge import general_intergrax_knowledge
from components.project_manager import project_manager
from components.ux_audit import ux_audit


# --- LLM adapter (Ollama) ---
adapter = LLMAdapterRegistry.create(
    name="ollama",
    chat=ChatOllama(model="llama3.1:latest"),
)

# --- Resources (optional) ---
resources = {
    # "retriever": intergraxRagRetriever(...),
    # "ranker": intergraxReRanker(...),
}

# --- Component registry ---
components = [
    general_intergrax_knowledge,
    ux_audit,
    final_summary,
    project_manager,
    final_summary,
    compliance_checker,
    cost_estimator,
]

embed_manager = IntergraxEmbeddingManager(
    verbose=True,
    provider="ollama",
    model_name="rjmalagon/gte-qwen2-1.5b-instruct-embed-f16:latest", 
    assume_ollama_dim=1536)

# --- Supervisor with fallback + final step ---
cfg = SupervisorConfig(
    llm_adapter=adapter,
    components=components,
    heuristic_enable=False,
    debug=True,
    pass_temperature=False,
    pass_max_tokens=False,
    resources=resources,

    # runtime fallback only when a step has no assigned component:
    fallback_component="General",
    fallback_on_miss=True,

    # two-stage planner (decompose -> per-step assignment)
    planner_mode="two_stage",
    plan_retries=2,
    assign_retries=2,
    assign_self_consistency=3,
    assign_threshold=0.50,          # start loosely, then increase to 0.6–0.7
    skip_on_low_confidence=False,   # still execute assignments even with low confidence
)

# --- Test query (single string) ---
query = (
    "Perform the following actions in the given order and under the given conditions: "
    "1. Based on the provided FIGMA mockups, act as a UX Auditor "
    "who analyzes the mockups for logic and functional correctness of the designed application. "
    "2. Verify whether the list of changes complies with the company's privacy policy and regulations. "
    "3. Prepare a summary report and present it to the Project Manager. "
    "4. If the PM rejects the project — provide a short summary and terminate. If they approve it, continue. "
    "5. Retrieve the financial report for the last quarter and evaluate whether the budget allows these changes; "
    "if the cost of changes is < 2 percent of the budget — approve. "
    "6. Finally, prepare a full report summarizing all steps, actions, and decisions."
)

supervisor = IntergraxSupervisor(cfg)
plan = supervisor.plan(query)
supervisor.print_plan(plan)
state = supervisor.execute_plan(plan=plan, query=query, verbose=True)


c:\ProgramData\anaconda3\envs\genai_longchain\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange



INTENT: procedural | Needs: rag=True, tools=True, general=True | Confidence: 1.00
1. Wcielić się w rolę UX Audytora
   Goal: Wcielić się w rolę UX Audytora
   How to execute: method=TOOL, depends_on=[]
   Component: Audytor UX
   Component found: True
   Assign score: 1.00 | low_confidence=False
   Inputs: ['przekazane makety FIGMY']
   Outputs: ['context_chunks', 'citations']
   Success criteria: ['wartościowe wyniki analizy']
   Fallback: Powtórzyć analizę
   Rationale:
     why_method: Domain operation handled by a specialized tool.
     why_component: Chosen by planner: Audytor UX
2. Sprawdzenie listy zmian
   Goal: Sprawdzenie listy zmian
   How to execute: method=RAG, depends_on=['S1']
   Component: Weryfikacja regulaminów i polityk prywatności
   Component found: True
   Assign score: 1.00 | low_confidence=False
   Inputs: ['lista zmian', 'polityka prywatności firmy', 'regulamin']
   Outputs: ['context_chunks', 'citations']
   Success criteria: ['wartościowe wyniki sprawdzenia'